# Example ML pipeline using BODC data

A simple overview for retrieving data from a croissant file and training a simple machine learning model.

## Notebook overview

This notebook loads a croissant file and trains a simple machine learning model using the darts library to forecast unseen bottom pressure recorder data. This is purely intended to be a proof of concept, to demonstrate that data can be obtained via a croissant file, and then used to train a machine learning model.

### Data
https://www.bodc.ac.uk/resources/inventories/edmed/report/155/ 

The data set comprises time series measurements from offshore pressure gauges mounted on the sea floor. The data holdings are approximately 250 observation months from 100 sites. The data have mainly been collected in the continental shelf seas around the British Isles. Data records contain date/time, total pressure and, occasionally, temperature. The sampling interval is typically 15 minutes or hourly, over deployment periods ranging from 1 to 6 months. Data were collected mainly by the Proudman Oceanographic Laboratory (POL), now the National Oceanography Centre (NOC) at Liverpool, and are managed by the British Oceanographic Data Centre (BODC).

In [ ]:
# import necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from io import BytesIO
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries
from darts.models import NBEATSModel


import mlcroissant as mlc

## Get & prepare data

In [ ]:
def find_flag(val):
    # find flagged data
    if str(val)[-1] in ["M"]:
        return 0
    return 1


def replace_bad(val):
    # parse flagged data
    if str(val)[-1] in ["M", "N"]:
        return np.nan
    return val


def make_datetime(d, t):
    # format separate columns as a datetime object
    dt = d + t
    return datetime.strptime(dt, "%Y/%m/%d%H.%M.%S")


def read_croissant(str_obj):
    # read croissant file and process as a pandas df
    # multiple variables in these files: we only want to get PPSCZZ01 data.

    if "PPSCZZ01" in str(str_obj):
        df = pd.read_csv(
            BytesIO(str_obj),
            skiprows=13,
            sep="[ ^]+",
            names=["row_num", "date", "time", "pressure"],
            engine="python",
        )
        df["qc_flag"] = df["pressure"].apply(find_flag)
        df["pressure"] = df["pressure"].apply(replace_bad).astype(float)
        df["datetime"] = df.apply(lambda x: make_datetime(x.date, x.time), axis=1)
        df["datetime"] = df["datetime"].dt.floor("Min")
        return df

In [ ]:
ds = mlc.Dataset("shelfbpr_bodc.json")
dfs = []
for f in ds.records(record_set="default"):
    out = read_croissant(f["dat/content"])
    if out is not None:
        dfs.append(out)

## Split into train and test sets

In [ ]:
time_series = []
i = 0
for df in dfs:
    try:
        df = TimeSeries.from_dataframe(
            df[["datetime", "pressure"]],
            time_col="datetime",
            freq="15min",
            value_cols="pressure",
            fillna_value=0,
        )
        df = df.resample(freq="15min", method="interpolate")[1:]
        time_series.append(df)
    except Exception:
        print(
            "skipping",
            i,
        )
    i = i + 1

## Produce TimeSeries objects ready for darts

In [ ]:
# Scale data too
scaler = Scaler()
scaled = scaler.fit_transform(time_series)

In [ ]:
train_test_split = 0.8

split = int(train_test_split * len(scaled))
train = scaled[:split]
test = scaled[split:]

## Train model

A GPU might be handy here (should be automatically detected): consider reducing the `train_test_split` if you're pushed for time.

In [ ]:
model = NBEATSModel(input_chunk_length=192, output_chunk_length=96, random_state=42)

model.fit(train, epochs=1, verbose=True)

In [ ]:
n = 192
test_x, test_y = [], []
for k in test:
    test_x.append(k[:-n])
    test_y.append(k[-n:])

## Measure performance on test set

In [ ]:
preds = model.predict(series=test_x, n=n)
cvtd = scaler.inverse_transform(preds)

In [ ]:
test_inverse = scaler.inverse_transform(test_x)
truth_inverse = scaler.inverse_transform(test_y)

In [ ]:
# plot some predictions
for item in range(len(test_inverse)):
    fig, ax = plt.subplots(figsize=(6, 3.5), layout="constrained")
    test_inverse[item][-2000:].plot(ax=ax, label="X")
    truth_inverse[item].plot(ax=ax, linestyle="-", label="Ground Truth", color="grey")
    cvtd[item].plot(ax=ax, label="Prediction", linestyle=":")

    fig.suptitle("Example (truncated) prediction")

## Calculate skill

In [ ]:
# Use RMSE to calculate prediction skill
from darts.metrics.metrics import rmse

# there's plenty others out there in various documentation: this is just an example from darts

rmses = rmse(truth_inverse, cvtd)
print(rmses)
